## **Kindle Review Sentiment Analyis.**

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv('all_kindle_review.csv')


1) Preprocessing And Cleaning

2) Train Test Split

3) BOW,TFIDF,Word2vec

4) Train ML algorithms

In [4]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [6]:
data=data[['reviewText','rating']]

In [8]:
data.shape

(12000, 2)

In [10]:
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [12]:
data['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [ ]:
## Preprocessing & cleaning

In [15]:
## Postive review which is 3 and above is ! and negative review less than 3 is 0
data['rating']=data['rating'].apply(lambda x: 0 if x < 3 else 1)

In [16]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [18]:
data['reviewText']=data['reviewText'].str.lower()

In [19]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [ ]:
## removing special

In [22]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from bs4 import BeautifulSoup


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
## Removing special characters
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s-]+', '', x))

## Remove the stopwords
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words('english')]))

## Remove URLs
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://[\w_-]+(?:\.[\w_-]+)+[\w.,@?^=%&:/~+#-]*', '', str(x)))

## Remove HTML tags
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

## Remove any additional spaces
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.split()))


In [27]:
data['reviewText']

,reviewText
0,jace rankin may short hes nothing mess man hau...
1,great short read didnt want put read one sitti...
2,ill start saying first four books wasnt expect...
3,aggie angela lansbury carries pocketbooks inst...
4,expect type book library pleased find price right
...,...
11995,valentine cupid vampire- jena ian another vamp...
11996,read seven books series apocalypticadventure o...
11997,book really wasnt cuppa situation man capturin...
11998,tried use charge kindle didnt even register ch...


In [28]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [31]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [33]:
data['reviewText'] = data['reviewText'].apply(lambda x: lemmatize_words(x))


In [34]:
## Train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data['reviewText'],data['rating'],test_size=0.2,random_state=42)

In [37]:
## using bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x_train_bow=cv.fit_transform(x_train).toarray()
x_test_bow=cv.transform(x_test).toarray()

In [38]:
##using tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x_train_tfidf=tfidf.fit_transform(x_train).toarray()
x_test_tfidf=tfidf.transform(x_test).toarray()

In [39]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

model_bow=GaussianNB()
model_bow.fit(x_train_bow,y_train)
y_pred=model_bow.predict(x_test_bow)


In [45]:
model_tfidf=GaussianNB()
model_tfidf.fit(x_train_tfidf,y_train)
y_pred=model_tfidf.predict(x_test_bow)


In [46]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [47]:
y_pred_bow=model_bow.predict(x_test_bow)
y_pred_tdidf=model_tfidf.predict(x_test_tfidf)
print('BOW')
print(confusion_matrix(y_test,y_pred_bow))
print(classification_report(y_test,y_pred_bow))
print(accuracy_score(y_test,y_pred_bow))
print('TFIDF')
print(confusion_matrix(y_test,y_pred_tdidf))
print(classification_report(y_test,y_pred_tdidf))
print(accuracy_score(y_test,y_pred_tdidf))

BOW
[[499 304]
 [717 880]]
              precision    recall  f1-score   support

           0       0.41      0.62      0.49       803
           1       0.74      0.55      0.63      1597

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.63      0.57      0.59      2400

0.5745833333333333
TFIDF
[[488 315]
 [696 901]]
              precision    recall  f1-score   support

           0       0.41      0.61      0.49       803
           1       0.74      0.56      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400

0.57875


In [48]:
!pip install gensim

import gensim

In [51]:
tokenized_train = [text.split() for text in x_train]
tokenized_test = [text.split() for text in x_test]

# Train the Word2Vec model
model = gensim.models.Word2Vec(sentences=tokenized_train, vector_size=100, window=5, min_count=1, workers=4)


word_vector = model.wv['good']
print(word_vector)



[ 7.4529842e-02  6.1027372e-01  7.4859548e-01  8.7830655e-02
  3.0621412e-01 -1.3329688e+00  9.7900492e-01  1.8558391e+00
 -8.5632378e-01 -9.5966184e-01 -3.6517758e-02 -1.4468074e+00
  3.2898256e-01  7.7682972e-01  8.2440331e-04 -8.3542663e-01
  7.5228256e-01 -7.6776803e-01 -3.3286744e-01 -2.2150199e+00
  6.4618029e-02  1.7889966e-01  6.5374148e-01 -5.4844719e-01
 -4.2540014e-01 -2.4485637e-01 -7.3537475e-01 -4.6285376e-01
 -1.4017820e+00 -1.7364055e-01  1.1267810e+00  9.8022379e-02
  1.4537635e-01 -7.3861647e-01 -1.3464894e+00  1.0723614e+00
  5.3500004e-02 -4.4285575e-01  5.5399188e-03 -1.2967439e+00
  3.5577753e-01 -9.0797305e-01  1.2150134e-01 -5.7898130e-02
  7.9882395e-01  1.0480341e-01 -1.1306207e+00 -8.2824945e-01
  7.9899901e-01  5.1262081e-01  8.5160136e-01 -9.0052098e-01
 -7.3901361e-01  1.6606963e-01 -3.9099929e-01  3.6352226e-01
  8.0788517e-01  1.3205026e-01 -1.2193962e+00  8.1822395e-02
 -3.6534075e-02 -2.6232475e-01 -1.8056219e-02  1.0788849e-01
 -1.2081543e+00  1.23055

In [53]:
def average_word2vec(doc):
    words = [word for word in doc.split() if word in model.wv.key_to_index]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in words], axis=0)


In [54]:
x_train_avg_w2v = np.array([average_word2vec(text) for text in x_train])
x_test_avg_w2v = np.array([average_word2vec(text) for text in x_test])


In [55]:
model_avg_w2v = GaussianNB()
model_avg_w2v.fit(x_train_avg_w2v, y_train)
y_pred_avg_w2v = model_avg_w2v.predict(x_test_avg_w2v)

print("Average Word2Vec Classifier Results:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_avg_w2v))
print("Classification Report:\n", classification_report(y_test, y_pred_avg_w2v))
print("Accuracy:", accuracy_score(y_test, y_pred_avg_w2v))

Average Word2Vec Classifier Results:
Confusion Matrix:
 [[ 621  182]
 [ 585 1012]]
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.77      0.62       803
           1       0.85      0.63      0.73      1597

    accuracy                           0.68      2400
   macro avg       0.68      0.70      0.67      2400
weighted avg       0.74      0.68      0.69      2400

Accuracy: 0.6804166666666667
